In [1]:
import boto3
from datetime import datetime

In [2]:
s3 = boto3.resource('s3')

In [3]:
BUCKET_NAME = "bucknellrobotics"
IMG_BUCKET = s3.Bucket(name=BUCKET_NAME)

In [4]:
def filename_parser(raw_key=None):
    filename = raw_key.split("/")[2] #This will get everything after "10_09_2019/ASIN/"
    return filename

In [5]:
def update_key(obj):
    filename = filename_parser(obj.key)
    new_key = "10_09_2019/" + filename
    return new_key
    #obj.put(Data=obj.data, Key=new_key)

In [6]:
def clean_obj(obj):
    # Approach 1
    obj.delete()
    
    # Approach 2
    #response = IMG_BUCKET.delete_objects( Delete={'Objects': [{'Key':obj.key},], 'Quiet':True},
    

In [7]:
def remove_from_folder(obj):
    '''
    This function will do all of the grunt work of 'moving' a file: 
    Make a new key, 
    copy the original object, change the copy's key, 
    and delete the original object
    '''
    old_key = obj.key
    new_key = update_key(obj)
    
    #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Bucket.copy
    copy_source = {'Bucket': BUCKET_NAME, 'Key': old_key}
    IMG_BUCKET.copy(copy_source, new_key)
    
    #Delete the original
    clean_obj(obj)
    

In [20]:
def main():
    start_time = datetime.time(datetime.now())
    i = 0
    k = 0
    for obj in IMG_BUCKET.objects.all():
        temp = obj.key.split("/")
        
        if (len(temp)>=3) and (temp[0] == "10_09_2019") and (len(temp[1]) == 10): 
        #There's a lot of other junk in the folder right now, I only want things in 10_09_2019 folder
            remove_from_folder(obj)

            i = (i + 1) % 1000
            if i == 0:
                k = k+1
                print(str(datetime.time(datetime.now())) + "   " + str(k) + "000 files fixed")

        
    end_time = datetime.time(datetime.now())
    print("\n\n" + str(end_time) + "  DONE")

In [21]:
main()

11:41:16.313605   1000 files fixed
11:45:41.797676   2000 files fixed
11:50:05.702549   3000 files fixed
11:54:27.709906   4000 files fixed
11:58:56.158105   5000 files fixed
12:03:39.504553   6000 files fixed
12:08:04.440584   7000 files fixed
12:12:34.685362   8000 files fixed
12:16:58.969771   9000 files fixed
12:21:31.597886   10000 files fixed
12:26:07.635746   11000 files fixed
12:30:29.384870   12000 files fixed
12:34:54.303597   13000 files fixed


12:46:07.184495  DONE


#### References: https://stackoverflow.com/questions/40336918/how-to-write-a-file-or-data-to-an-s3-object-using-boto3 
#### https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Object.put

#### I'm seeing 3 possiblilities for deleting objects. 
#### 1) https://stackoverflow.com/questions/3140779/how-to-delete-files-from-amazon-s3-bucket  -  The one that I'm using. Just obj.delete()
#### 2) https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.delete_object  -  Seems more specific but is client based, not resource
#### 3) https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Bucket.delete_objects  -  Can delete up to 1000 objects at a time, reduces overhead. Could be useful but its not how I'm doing this. Resource, not client.

#### Looking into a copy method. https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Bucket.copy

